In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Load data
df = pd.read_csv(r"C:\Users\khush\OneDrive\Documents\features.csv")

# Use 'Temperature' and 'CPI' as features (drop rows with missing values in either)
df = df[['Temperature', 'CPI','Unemployment']].dropna()

# Normalize features
scaler = MinMaxScaler()
data_scaled = scaler.fit_transform(df)

# Sequence creation function
def create_sequences(data, seq_length):
    X, y = [], []
    for i in range(len(data) - seq_length):
        X.append(data[i:i+seq_length])
        y.append(data[i+seq_length][0])  # Predict Temperature (1st column)
    return np.array(X), np.array(y)

# Sequence lengths to experiment with
sequence_lengths = [7, 14, 30, 60]
results = []

for seq_len in sequence_lengths:
    print(f"\n🔄 Training with sequence length: {seq_len}")

    # Generate sequences
    X, y = create_sequences(data_scaled, seq_len)

    # LSTM expects input shape: (samples, time_steps, features)
    X = X.reshape((X.shape[0], X.shape[1], X.shape[2]))

    # Train/test split
    split = int(0.8 * len(X))
    X_train, X_test = X[:split], X[split:]
    y_train, y_test = y[:split], y[split:]

    # LSTM model
    model = Sequential()
    model.add(LSTM(64, activation='relu', input_shape=(seq_len, X.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Train model
    model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))

    print(f"📉 MAE: {mae:.4f} | RMSE: {rmse:.4f}")
    results.append((seq_len, mae, rmse))

# Results DataFrame
results_df = pd.DataFrame(results, columns=["Sequence Length", "MAE", "RMSE"])
print("\n📊 Sequence Length Experiment Results:")
print(results_df)

# Plot
plt.figure(figsize=(10,5))
plt.plot(results_df["Sequence Length"], results_df["MAE"], marker='o', label='MAE')
plt.plot(results_df["Sequence Length"], results_df["RMSE"], marker='o', label='RMSE')
plt.xlabel("Sequence Length (Time Window)")
plt.ylabel("Error")
plt.title("Effect of Sequence Length on Temperature Forecasting (with CPI)")
plt.legend()
plt.grid(True)
plt.show()